In [2]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import time
import re
ash_excitation = []

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity


#Hamiltonian 
symbols = ["H","H","H","H"]
r_bohr = 5.0*1.8897259886  #5.0 Angstrom in Bohr 
coordinates = np.array([[0.0,0.0, 1*r_bohr], [0.0, 0.0, 2*r_bohr], [0.0,0.0,3*r_bohr],[0.0, 0.0, 4*r_bohr]])
# Calculation of the Hamiltonian
H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-6g", method="pyscf")


electrons = 4
orbitals = 8
# Calculation of the excitation operators
singles, doubles = qml.qchem.excitations(electrons, orbitals,fermionic=False)
print('Singles are',singles)
print('Doubles are',doubles)
hf_state = qchem.hf_state(electrons, qubits)
print(f"Total number of excitations = {len(singles) + len(doubles)}")

#Calculation of HF state
dev = qml.device("default.qubit", wires=qubits)
@qml.qnode(dev)
def circuit(hf_state, electrons, qubits, H):
    # Prepare the Hartree-Fock state
    print('Updated hf_state is', hf_state)  #HF state : [1,1,1,1,0,0,0,0] - [alpha, beta, alpha, beta, alpha, beta, alpha, beta]
    qml.BasisState(hf_state, wires=range(qubits))

    return qml.expval(H)   #Calculating the expectation value of the Hamiltonian
print('HF state is', circuit(hf_state, electrons, qubits, H))

print('The original hamiltonian is', H)
#Putting all the functions here
#1.1091571486954503

Singles are [[0, 4], [0, 6], [1, 5], [1, 7], [2, 4], [2, 6], [3, 5], [3, 7]]
Doubles are [[0, 1, 4, 5], [0, 1, 4, 7], [0, 1, 5, 6], [0, 1, 6, 7], [0, 2, 4, 6], [0, 3, 4, 5], [0, 3, 4, 7], [0, 3, 5, 6], [0, 3, 6, 7], [1, 2, 4, 5], [1, 2, 4, 7], [1, 2, 5, 6], [1, 2, 6, 7], [1, 3, 5, 7], [2, 3, 4, 5], [2, 3, 4, 7], [2, 3, 5, 6], [2, 3, 6, 7]]
Total number of excitations = 26
Updated hf_state is [1 1 1 1 0 0 0 0]
HF state is -1.216023859130329
The original hamiltonian is -1.1091571486954515 * I(0) + 0.04196325905665746 * Z(0) + 6.339400653837491e-05 * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4)) + 6.339400653837491e-05 * (X(0) @ Z(1) @ Z(2) @ Z(3) @ X(4)) + 0.04183466728352421 * Z(2) + 0.015433496464646382 * (Z(0) @ Z(2)) + 6.344558586986833e-05 * (Y(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6)) + 6.344558586986833e-05 * (X(2) @ Z(3) @ Z(4) @ Z(5) @ X(6)) + -0.004378038402039231 * (Z(0) @ Y(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6)) + -0.004378038402039231 * (Z(0) @ X(2) @ Z(3) @ Z(4) @ Z(5) @ X(6)) + 0.04170505593332356 * 

In [19]:
# Commutator calculation for HF state
dev = qml.device("default.qubit", wires=qubits)
@qml.qnode(dev,diff_method='backprop')
def commutator_0(H,w, k):  #H is the Hamiltonian, w is the operator, k is the basis state - HF state
    qml.BasisState(k, wires=range(qubits))
    res = qml.commutator(H, w)   #Calculating the commutator
    #res = (qml.prod(H, w)) - (qml.prod(w, H))
    return qml.expval(res)

# Commutator calculation for other states except HF state
dev = qml.device("default.qubit", wires=qubits)
@qml.qnode(dev,diff_method='backprop')
def commutator_1(H,w, k): #H is the Hamiltonian, w is the operator, k is the basis state
    qml.StatePrep(k, wires=range(qubits))
    res = qml.commutator(H, w) #Calculating the commutator
    return qml.expval(res)

# Energy calculation 
dev = qml.device("default.qubit", wires=qubits)
@qml.qnode(dev, diff_method='backprop')
def ash(params, ash_excitation, hf_state, qubits, H):
    #qml.BasisState(hf_state, wires=range(qubits))
    [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]  #Appln of HF state
    for i, excitation in enumerate(ash_excitation):
        if len(ash_excitation[i]) == 4:
            qml.DoubleExcitation(params[i], wires = ash_excitation[i])  #Applying the double excitation and their parameters
        elif len(ash_excitation[i]) == 2:
            print('Single Exc going in is', ash_excitation[i])
            qml.SingleExcitation(params[i], wires = ash_excitation[i]) #Applying the single excitation and their parameters
            
    return qml.expval(H)  #Calculating the expectation value of the Hamiltonian

# Calculation of New state, same as the above function but with the state return

dev = qml.device("default.qubit", wires=qubits)
@qml.qnode(dev, diff_method='backprop')
def new_state(hf_state, ash_excitation, fparams, params):
    #qml.BasisState(hf_state, wires=range(qubits))
    [qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #Applying the HF state
    for i, excitations in enumerate(ash_excitation):
        if len(ash_excitation[i]) == 4:
            print('Exc. dealing right now is', ash_excitation[i])
            print('The params that are going in', params[i])
            qml.DoubleExcitation(params[i], wires = ash_excitation[i])
        elif len(ash_excitation[i]) == 2:
            print('Single Exc. dealing right now is', ash_excitation[i])
            print('Single exc params that are going in', params[i])
            qml.SingleExcitation(params[i], wires = ash_excitation[i])
            

    return qml.state()

In [20]:

electrons = 4
orbitals = 8
singles, doubles = qml.qchem.excitations(electrons, orbitals, fermionic=True)
optimizer = qml.GradientDescentOptimizer(stepsize=0.5)  #Optimizer

print('The Hamiltonian is ', H)
adit = 6
fparams = []
excitations= []
operator_pool = (singles) + (doubles)  #Operator pool - Singles and Doubles
states = [hf_state]


for j in range(adit):
    print('The adapt iteration now is', j)  #Adapt iteration
    max_value = float('-inf')
    max_operator = None
    # Start with the most recent state (last state in the list)
    k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state

    for i in operator_pool:
        print('The current excitation operator is', i)   #Current excitation operator - fermionic one
        w = qml.fermi.jordan_wigner(i)  #JW transformation
        if np.array_equal(k, hf_state): # If the current state is the HF state
            print('Print, if this is activated - HF state')
            current_value = abs(2*(commutator_0(H, w, k)))      #Commutator calculation is activated  
        else:
            current_value = abs(2*(commutator_1(H, w, k)))      #For other states, commutator calculation is activated
        print(f'The expectation value of {i} is', current_value)

        if current_value > max_value:
            max_value = current_value
            max_operator = i

    print(f"The highest operator value is {max_value} for operator {max_operator}")  #Highest operator value

    # Convert operator to excitations and append to ash_excitation
    indices_str = re.findall(r'\d+', str(max_operator))
    excitations = [int(index) for index in indices_str]
    print('Highest gradient excitation is', excitations)
    ash_excitation.append(excitations) #Appending the excitations to the ash_excitation
    print('The current status of ash_excitation is', ash_excitation)
    print('Moving towards parameters')
    params = np.zeros(len(ash_excitation), requires_grad=True)  #Parameters initialization
    print('The length of parameters is', len(params))
    
    # Cost function definition
    cost_fn = qml.QNode(ash, dev, interface="autograd", diff_method="backprop") #Cost function definition, gradient calculated automatically and method is backprop
    print('Going to do energy calculation')
    for n in range(150):
        print(f'Each step, the iteration is {n} and the parameter is {params}')
        params, energy = optimizer.step_and_cost(cost_fn, params, ash_excitation=ash_excitation, hf_state=hf_state, qubits=qubits, H=H)
        if n % 5 == 0:
            print(f"step = {n}, E = {energy:.8f} Ha")
    fparams.append(params)
    print('Updated params are', params)
    print('Updated excitation are', ash_excitation)
    
    # New state generation - With the updated parameters
    ostate = new_state(hf_state, ash_excitation, fparams, params)
    print(qml.draw(new_state, level="device", max_length=100)(hf_state,ash_excitation,fparams,params))
    print('The updated state is', ostate)
    # Append the new state to the states list
    states.append(ostate)  


The Hamiltonian is  -1.109157148695451 * I(0) + 0.04196325905665785 * Z(0) + 6.339400653744857e-05 * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4)) + 6.339400653744857e-05 * (X(0) @ Z(1) @ Z(2) @ Z(3) @ X(4)) + 0.041834667283523475 * Z(2) + 0.015433496464716076 * (Z(0) @ Z(2)) + 6.344558587105921e-05 * (Y(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6)) + 6.344558587105921e-05 * (X(2) @ Z(3) @ Z(4) @ Z(5) @ X(6)) + -0.004378038402061964 * (Z(0) @ Y(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6)) + -0.004378038402061964 * (Z(0) @ X(2) @ Z(3) @ Z(4) @ Z(5) @ X(6)) + 0.04170505593332312 * Z(4) + 0.01984336499843015 * (Z(0) @ Z(4)) + 0.04157626163217726 * Z(6) + 0.022112211956202707 * (Z(0) @ Z(6)) + 0.04196325905665777 * Z(1) + 0.06277200433835764 * (Z(0) @ Z(1)) + 0.00250256404763367 * (Y(0) @ Z(2) @ Z(3) @ Y(4)) + 0.00250256404763367 * (X(0) @ Z(2) @ Z(3) @ X(4)) + 0.047312950419660486 * (Y(0) @ X(1) @ X(2) @ Y(3)) + -0.047312950419660486 * (Y(0) @ Y(1) @ X(2) @ X(3)) + -0.047312950419660486 * (X(0) @ X(1) @ Y(2) @ Y(3)) + 0.04731

/Users/cwq/CodingwithQiskit/miniconda3/envs/cwq/lib/python3.12/site-packages/autoray/autoray.py:1127: ComplexWarning: Casting complex values to real discards the imaginary part
  return x.astype(dtype, **kwargs)


The expectation value of a⁺(0) a(4) is 1.5771399313564347e-05
The current excitation operator is a⁺(0) a(6)
Print, if this is activated - HF state
The expectation value of a⁺(0) a(6) is 0.0
The current excitation operator is a⁺(1) a(5)
Print, if this is activated - HF state
The expectation value of a⁺(1) a(5) is 1.5771399313567816e-05
The current excitation operator is a⁺(1) a(7)
Print, if this is activated - HF state
The expectation value of a⁺(1) a(7) is 0.0
The current excitation operator is a⁺(2) a(4)
Print, if this is activated - HF state
The expectation value of a⁺(2) a(4) is 0.0
The current excitation operator is a⁺(2) a(6)
Print, if this is activated - HF state
The expectation value of a⁺(2) a(6) is 1.5613748771391503e-05
The current excitation operator is a⁺(3) a(5)
Print, if this is activated - HF state
The expectation value of a⁺(3) a(5) is 0.0
The current excitation operator is a⁺(3) a(7)
Print, if this is activated - HF state
The expectation value of a⁺(3) a(7) is 1.561374

In [15]:
print(qml.specs(cost_fn)(params, ash_excitation, hf_state, qubits, H))
ash_excitation

Single Exc going in is [0, 4]
Single Exc going in is [3, 7]
{'resources': Resources(num_wires=8, num_gates=10, gate_types=defaultdict(<class 'int'>, {'PauliX': 4, 'DoubleExcitation': 4, 'SingleExcitation': 2}), gate_sizes=defaultdict(<class 'int'>, {1: 4, 4: 4, 2: 2}), depth=6, shots=Shots(total_shots=None, shot_vector=())), 'errors': {}, 'num_observables': 1, 'num_diagonalizing_gates': 1, 'num_trainable_params': 6, 'num_device_wires': 8, 'num_tape_wires': 8, 'device_name': 'default.qubit', 'level': 'gradient', 'gradient_options': {}, 'interface': 'auto', 'diff_method': 'backprop', 'gradient_fn': 'backprop'}


[[2, 3, 6, 7], [0, 3, 5, 6], [0, 1, 4, 5], [1, 2, 4, 7], [0, 4], [3, 7]]

## Implementing scipy Minimize

In [3]:
from scipy.optimize import minimize



dev = qml.device('default.qubit', wires=8)
#ash_excitation = [[2, 3, 6, 7], [0, 3, 5, 6], [0, 1, 4, 5], [1, 2, 4, 7], [2, 6]]
ash_excitation =[[2, 3, 6, 7], [0, 3, 5, 6], [0, 1, 4, 5], [1, 2, 4, 7], [0, 4]]

@qml.qnode(dev)
def circuit(ash_excitation, hf_state, params, H):
    [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]
    for i, excitation in enumerate(ash_excitation):
        if len(ash_excitation[i]) == 4:
            qml.DoubleExcitation(params[i], wires = ash_excitation[i])
        elif len(ash_excitation[i]) == 2:
            #print('Single Exc going in is', ash_excitation[i])
            qml.SingleExcitation(params[i], wires = ash_excitation[i])
            
    return qml.expval(H)


def cost(params):
    energy = circuit(ash_excitation, hf_state, params, H)
    return energy

params = np.array([0.99, 0.012, 0.05, 0.5,1.0], requires_grad=True)

minimize(cost, params, method='BFGS')

  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: -1.8554137514569642
        x: [ 8.444e-01  1.366e+00  1.050e+00  1.227e+00 -2.658e-05]
      nit: 16
      jac: [-3.725e-07 -1.639e-07  3.874e-07  1.028e-06  1.937e-07]
 hess_inv: [[ 2.952e+00  1.493e-01 ... -8.446e-01  4.542e-01]
            [ 1.493e-01  3.390e+00 ... -2.348e-01 -2.633e-02]
            ...
            [-8.446e-01 -2.348e-01 ...  4.707e+00 -3.919e-01]
            [ 4.542e-01 -2.633e-02 ... -3.919e-01  9.083e+00]]
     nfev: 108
     njev: 18

## Jax interface - Optax

In [4]:
import jax
from jax import numpy as jnp
import pennylane as qml
import optax
import jax.numpy as jnp
from pennylane import qchem
from pennylane import numpy as np

jax.config.update("jax_enable_x64", True)


X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity

symbols = ["H","H","H","H"]
r_bohr = 5.0*1.8897259886
coordinates = np.array([[0.0,0.0, 1*r_bohr], [0.0, 0.0, 2*r_bohr], [0.0,0.0,3*r_bohr],[0.0, 0.0, 4*r_bohr]])
H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-6g", method = "pyscf")

print('The original hamiltonian is', H)
electrons = 4
orbitals = 8
qubits = 8
singles, doubles = qml.qchem.excitations(electrons, orbitals,fermionic=True)
print('Singles are',singles)
print('Doubles are',doubles)
hf_state = qchem.hf_state(electrons, qubits)
print(f"Total number of excitations = {len(singles) + len(doubles)}")



dev = qml.device("default.qubit", wires=qubits)
@qml.qnode(dev)
def circuit(hf_state, electrons, qubits, H):
    # Prepare the Hartree-Fock state
    print('Updated hf_state is', hf_state)
    qml.BasisState(hf_state, wires=range(qubits))
    #[qml.PauliX(i) for i in jnp.nonzero(hf_state)[0]] #HF state

    return qml.expval(H)
print('HF state is', circuit(hf_state, electrons, qubits, H))

#dev = qml.device("default.qubit", wires=qubits)

#@jax.jit
#@qml.qnode(dev, interface="jax")
#def circuit(param):
#    qml.RX(param, wires=0)
##    qml.CNOT(wires=[0, 1])
#    return qml.expval(H)

#ash_excitation = [[2,6],[1, 2, 4, 7],[0,1,4,5],[0, 3, 5, 6],[2, 3, 6, 7]]
#ash_excitation =  [[3, 7], [0, 4], [1, 2, 4, 7], [0, 1, 4, 5], [0, 3, 5, 6], [2, 3, 6, 7]]
ash_excitation =  [[2,3,6,7], [0,3,5,6], [0,1,4,5], [1,2,4,7], [2,6]]

optimizer = optax.adam(learning_rate=0.1)
#optimizer = optax.lbfgs()
params = jnp.zeros(len(ash_excitation), dtype=jnp.float32)
opt_state = optimizer.init(params)
print('The parameters that are going in ', len(params))

dev = qml.device("default.qubit", wires=qubits)

# QNode to evaluate energy
@qml.qnode(dev)
#@jax.jit
@qml.qnode(dev, interface="jax")
def circuit(params, ash_excitation, hf_state, qubits, H):
    qml.BasisState(hf_state, wires=range(qubits))
    for i, excitations in enumerate(ash_excitation):
        if len(excitations) == 4:
            #Exchanged the order of wires 1 and wires 2
            #qml.FermionicDoubleExcitation(weight = params[i], wires2=excitations[2:][::-1], wires1=excitations[:2][::-1])
            qml.FermionicDoubleExcitation(weight = params[i], wires1=excitations[2:][::-1], wires2=excitations[:2][::-1])
        else:
            #print(f'Single excitations coming in is {excitations} and parameters are {params[i]}')
            qml.FermionicSingleExcitation(weight= params[i], wires = excitations)
    return qml.expval(H)

def cost_fn(params, ash_excitation, hf_state, qubits, H):
    # Compute the energy
    energy = circuit(params, ash_excitation, hf_state, qubits, H)
    return energy

def minimize_function(params):
    energy= cost_fn(params, ash_excitation, hf_state, qubits, H)
    return energy

gradient_tolerance = 1e-8 
gradient_max_threshold = 1e-8
for i in range(250):
    #cost, grad_circuit = jax.value_and_grad(circuit)(params)
    cost, grad_circuit = jax.value_and_grad(minimize_function)(params)
        # Compute the norm of the gradient
    grad_norm = jnp.linalg.norm(grad_circuit)
    grad_max = jnp.max(jnp.abs(grad_circuit))


    # Print the current gradient norm (optional for debugging)
    print(f"Step {i}, cost: {cost}, grad_norm: {grad_norm}, grad_max: {grad_max}")

    # Check if the gradient norm is below the tolerance
    if grad_norm < gradient_tolerance:
        print(f"Convergence achieved at step {i}. Gradient norm {grad_norm} is below tolerance.")
        break
    if grad_max < gradient_max_threshold:
        print(f"Convergence achieved at step {i}. Maximum gradient {grad_max} is below the threshold.")
        break 
    updates, opt_state = optimizer.update(grad_circuit, opt_state)
    params = optax.apply_updates(params, updates)
    #print(f"step {i}, cost {cost}")
print('Final energy is', cost)

The original hamiltonian is -1.1091571486954508 * I(0) + 0.04196325905665828 * Z(0) + 6.339400653668963e-05 * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4)) + 6.339400653668963e-05 * (X(0) @ Z(1) @ Z(2) @ Z(3) @ X(4)) + 0.04183466728352326 * Z(2) + 0.015433496464767268 * (Z(0) @ Z(2)) + 6.344558587176785e-05 * (Y(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6)) + 6.344558587176785e-05 * (X(2) @ Z(3) @ Z(4) @ Z(5) @ X(6)) + -0.004378038402074129 * (Z(0) @ Y(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6)) + -0.004378038402074129 * (Z(0) @ X(2) @ Z(3) @ Z(4) @ Z(5) @ X(6)) + 0.04170505593332266 * Z(4) + 0.019843364998430146 * (Z(0) @ Z(4)) + 0.04157626163217762 * Z(6) + 0.02211221195620431 * (Z(0) @ Z(6)) + 0.0419632590566583 * Z(1) + 0.06277200433838766 * (Z(0) @ Z(1)) + 0.0025025640478904244 * (Y(0) @ Z(2) @ Z(3) @ Y(4)) + 0.0025025640478904244 * (X(0) @ Z(2) @ Z(3) @ X(4)) + 0.047312950419634826 * (Y(0) @ X(1) @ X(2) @ Y(3)) + -0.047312950419634826 * (Y(0) @ Y(1) @ X(2) @ X(3)) + -0.047312950419634826 * (X(0) @ X(1) @ Y(2) @ Y(3)